In [ ]:
!pip install hsclient
!pip install numpy
!pip install scipy
!pip install requests

In [240]:
import pickle
targets = ["beta.hydroshare.org"]
meta_run_number = 0 # manually change for first run/ second
if meta_run_number == 0:
    meta_runs = [{}, {}] # pre and post
else:
    dbfile = open('pickle', 'rb')    
    meta_runs = pickle.load(dbfile)
    dbfile.close()
    
clients = {}
filename = "local-build-discover.sh"
diffs= {}

In [267]:
import logging
logger = logging.getLogger('django.timer')
logger.setLevel(logging.INFO)
handler = logging.FileHandler(filename="log/timer.log")
formatter = logging.Formatter("%(asctime)s - "
                              "%(levelname)s - "
                              "%(funcName)s - "
                              "line %(lineno)s - "
                              "%(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

In [273]:
def create_timer_log(target, function_name=None, scheme="https://"):
    import requests
    import uuid
    import inspect

    if not function_name:
        function_name = inspect.stack()[1][3]

    uuid = uuid.uuid4()
    url = f"{scheme}{target}/hsapi/timer/{function_name}/{uuid}"
    r = requests.get(url)
    if not r.ok:
        message = f"FAILED creating time log for url:\n{url}"
        logger.info(message)
        print(message)
    else:
        message = r.json()["message"]
        logger.info(message)
        print(message)

# create_timer_log("localhost:8000", "test_naem", scheme="http://")

In [241]:
from hsclient import HydroShare
for target in targets:
    hs = HydroShare(host=target, port=443, protocol='https')
    hs.sign_in()
    hs.resources = []
    diffs[target] = []
    clients[target] = hs

In [278]:
from time import time
from functools import wraps
import numpy as np
import scipy
from copy import deepcopy

def timer_func(func): 
    @wraps(func)
    def wrap_func(*args, **kwargs):
        target = kwargs['target']
        # create_timer_log(target, func.__name__)
        t1 = time() 
        result = func(*args, **kwargs)
        t2 = time()
        diff = t2-t1
        diffs[target].append(diff)
        message = f'Function {func.__name__!r} executed in {(diff):.4f}s'
        logger.info(message)
        print(message) 
        return result 
    return wrap_func


def create_res(hs, time=True, target=None):
    @timer_func
    def create_res_time(hs=None, target=None):
        resource = hs.create()
        hs.resources.append(resource)
    
    if time:
        create_timer_log(target)
        create_res_time(hs=hs, target=target)
    else:
        resource = hs.create()
        hs.resources.append(resource)

@timer_func
def delete_resources(hs=None, target=None):
    create_timer_log(target)
    for res in hs.resources:
        res.delete()

@timer_func
def download_all_resources(hs=None, target=None):
    create_timer_log(target)
    for res in hs.resources:
        try:
            res.download()
        except Exception as e:
            print(e)

def download_single_resource(hs=None, target=None):
    create_timer_log(target)
    create_res(hs, time=False, target=target)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def download_single_resource_inner(hs=res, target=None):
        res.download()

    download_single_resource_inner(hs=res, target=target)

def delete_single_resource(hs=None, target=None):
    create_timer_log(target)
    create_res(hs, time=False, target=target)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def delete_single_resource_inner(hs=res, target=None):
        res.delete()

    delete_single_resource_inner(hs=res, target=target)

def upload_file(hs, target=None):
    create_timer_log(target)
    create_res(hs, time=False, target=target)
    res = hs.resources[-1]

    @timer_func
    def upload_file_inner(hs=res, target=None):
        res.file_upload(filename)

    upload_file_inner(hs=res, target=target)


def create_dir_upload_file(hs=None, target=None):
    create_timer_log(target)
    create_res(hs, time=False, target=target)
    res = hs.resources[-1]

    @timer_func
    def create_dir_upload_file_inner(hs=res, target=None):
        res.folder_create('New_Folder')
        res.file_upload(filename, destination_path='New_Folder')
    create_dir_upload_file_inner(hs=res, target=target)

def upload_and_movefile(hs=None, target=None):
    create_timer_log(target)
    create_res(hs, time=False, target=target)
    res = hs.resources[-1]
    
    @timer_func
    def upload_and_movefile_inner(hs=res, target=None):
        res.folder_create('New_Folder')
        res.file_upload(filename, destination_path='New_Folder')
        file = res.file(path=f"New_Folder/{filename}]")
        res.file_rename(file, filename)
    upload_and_movefile_inner(hs=res, target=target)

def download_single_file(hs=None, target=None):
    create_timer_log(target)
    create_res(hs, time=False, target=target)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def download_single_file_inner(hs=res, target=target):
        file = res.file(path=filename)
        res.file_download(file)
    download_single_file_inner(hs=res, target=target)

def delete_single_file(hs=None, target=None):
    create_timer_log(target)
    create_res(hs, time=False, target=target)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def delete_single_file_inner(hs=res, target=None):
        file = res.file(path=filename)
        res.file_delete(file)
    delete_single_file_inner(hs=res, target=target)
        

def run_comparisons(functions_to_run, runs=1):
    for funct in functions_to_run:
        message = f"********* Start {funct.__name__!r} *********"
        print(message)
        logger.info(message)
        for i in range(runs):
            for target in targets:
                client = clients[target]
                message = f"Starting run {i} of {funct.__name__} on {target}..."
                print(message)
                logger.info(message)
                funct(hs=client, target=target)
                print(diffs[target])
                logger.info(diffs[target])
        print("\n\n")
        # Store the diffs for later
        meta_runs[meta_run_number][funct.__name__] = deepcopy(diffs)
        for target in targets:
            diffs[target].clear()

def clear_resources():
    for target in targets:
        client = clients[target]
        client.resources = []
        diffs[target].clear()
  

In [279]:
# meta_runs = [{}, {}]

In [280]:
clear_resources()
run_comparisons([download_single_resource, create_dir_upload_file, delete_single_file ], runs=3)
run_comparisons([delete_resources], runs=1)

********* Start 'download_single_resource' *********
Starting run 0 of download_single_resource on beta.hydroshare.org...
Starting call function download_single_resource at 2023-12-19 18:49:50 with UUID=19db6860-c2ba-47b9-8454-7547a72c2b2d
Function 'download_single_resource_inner' executed in 33.2561s
[33.25609993934631]
Starting run 1 of download_single_resource on beta.hydroshare.org...
Starting call function download_single_resource at 2023-12-19 18:50:50 with UUID=9b9899ae-1bb9-4b68-af15-7c2623fe92e2
Function 'download_single_resource_inner' executed in 37.1817s
[33.25609993934631, 37.18172574043274]
Starting run 2 of download_single_resource on beta.hydroshare.org...
Starting call function download_single_resource at 2023-12-19 18:51:55 with UUID=3e1b90ae-ccd9-4a38-aa98-accd096e206b
Function 'download_single_resource_inner' executed in 39.6335s
[33.25609993934631, 37.18172574043274, 39.633548974990845]



********* Start 'create_dir_upload_file' *********
Starting run 0 of create_

In [281]:
clear_resources()
run_comparisons([upload_file], runs=10)

********* Start 'upload_file' *********
Starting run 0 of upload_file on beta.hydroshare.org...
Starting call function upload_file at 2023-12-19 19:03:15 with UUID=317727c8-9ccb-4df9-80af-6f5999a443c6
Function 'upload_file_inner' executed in 5.7137s
[5.71367883682251]
Starting run 1 of upload_file on beta.hydroshare.org...
Starting call function upload_file at 2023-12-19 19:03:42 with UUID=5366be82-44eb-4bb5-9d0f-bccf311b0be5
Function 'upload_file_inner' executed in 3.0886s
[5.71367883682251, 3.0885658264160156]
Starting run 2 of upload_file on beta.hydroshare.org...
Starting call function upload_file at 2023-12-19 19:04:16 with UUID=c0dc4971-d006-4bd2-81e8-a9e2da599650
Function 'upload_file_inner' executed in 5.8259s
[5.71367883682251, 3.0885658264160156, 5.8258891105651855]
Starting run 3 of upload_file on beta.hydroshare.org...
Starting call function upload_file at 2023-12-19 19:04:50 with UUID=e91bf227-0dd3-425e-bf7b-d2084cace994
Function 'upload_file_inner' executed in 2.7734s
[5.

In [282]:
clear_resources()
run_comparisons([create_res, download_all_resources, delete_single_resource], runs=5)

********* Start 'create_res' *********
Starting run 0 of create_res on beta.hydroshare.org...
Starting call function create_res at 2023-12-19 19:08:32 with UUID=b5a8919e-44ab-4722-b5d0-b94cfa64e572
Function 'create_res_time' executed in 28.1364s
[28.136365175247192]
Starting run 1 of create_res on beta.hydroshare.org...
Starting call function create_res at 2023-12-19 19:09:00 with UUID=da23d7d3-3706-4ae7-9c86-3ab32e81fad9
Function 'create_res_time' executed in 28.3560s
[28.136365175247192, 28.356043577194214]
Starting run 2 of create_res on beta.hydroshare.org...
Starting call function create_res at 2023-12-19 19:09:29 with UUID=61720e2d-1129-4bce-813f-66a34007a36b
Function 'create_res_time' executed in 26.3650s
[28.136365175247192, 28.356043577194214, 26.364990949630737]
Starting run 3 of create_res on beta.hydroshare.org...
Starting call function create_res at 2023-12-19 19:09:56 with UUID=1c749e46-59f3-40ad-984a-fc9a58c11c3c
Function 'create_res_time' executed in 30.0325s
[28.136365

In [283]:
import pickle
file = open('pickle', 'ab')
    
# source, destination
pickle.dump(meta_runs, file)                    
file.close()

In [ ]:
# TODO: these ones still seem not to work...
#clear_resources()
#run_comparisons([upload_and_movefile, download_single_file ], runs=2)

In [ ]:
def run_stats():
    for func in meta_runs[0].keys():
        for idx, meta_run in enumerate(meta_runs):
            print(f"Analyzing function {func} on meta run number {idx}")
            target_obj = meta_run[func]
            for target, runs in target_obj.items():
                print(f"Average time for {func} on {target}: {sum(runs)/len(runs)}")
                print(f"Devation for {func} on {target}: {np.std(runs)}")
            if len(target_obj.keys()) == 2:
                print(f"Comparing runs of {func} between targets {target_obj.keys()}")
                # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
                result = scipy.stats.ttest_ind(target_obj.values()[0], target_obj.values()[1])
                print(result)
        if len(meta_runs) == 2:
            print("Comparing between two meta runs")
            if func in meta_runs[0] and func in meta_runs[1]:
                for target in meta_runs[0][func].keys():
                    print(f"Comparing meta runs for target {target}")
                    result = scipy.stats.ttest_ind(meta_runs[0][func][target], meta_runs[1][func][target])
                    print(result)
            else:
                print(f"{func} values are missing from one of the meta runs")
        else:
            print("Meta run number not = 2 so can't compare between meta runs")
        print("\n\n")

In [ ]:
run_stats()